In [1]:
import numpy as np
from activityFunction import ActivationFunction as AF
from errorFunction import ErrorFunction as EF

In [17]:
class nn:
    
    def __init__(self, af='Sigmoid', ef='cross_entropy', lr=0.5 ):
        
        self.af = AF(types=af)
        self.ef = EF(types=ef)
        self.lr = lr
        self.layer_state = np.array([])
    
    def add_layer(self, input_size, neuron_size ):
        
        state = Layer_state(input_size, neuron_size, self.af)
        self.layer_state = np.append(self.layer_state, state) 
        
    def feedforward(self, inputs):
        
        print(inputs)
        print(inputs.shape)
        data = np.array(inputs[0, 0 : self.layer_state[0].inputs ])
        for i, Layer in np.ndenumerate(self.layer_state):
                
            data = Layer.feed_forward(data)
            
        return data
    
    
    def calculate_delta(self,output):
        
        # last delta = derivative error function * derivative activity function
#         self.layer_state[-1].delta = self.af.dfunc(self.layer_state[-1].output) * self.ef.dfunc(output, self.layer_state[-1].output) 
        self.layer_state[-1].delta = self.layer_state[-1].output - output 
        i = np.size(self.layer_state) - 2 # sub 2, because have already calcutaled the last delta
        
        while i >= 0:
            
            # delta(i) = weight * delta(i+1) * derivative activity function
#             self.layer_state[i].delta = self.layer_state[i+1].weight.T.dot( self.layer_state[i+1].delta ) * self.af.dfunc(self.layer_state[i].output)
            self.layer_state[i].delta = self.layer_state[i + 1].weight.T.dot(self.layer_state[i+1].delta) * self.layer_state[i].output * (1 - self.layer_state[i].output)
            i -= 1
            
    def update_weights(self, inputs):
        
        # update first weight and bias
        ## weight = weight - learning * delta * input
        ## bias = bias - learning rate * delta
        self.layer_state[0].weight = self.layer_state[0].weight - self.lr * self.layer_state[0].delta.dot(inputs)
        self.layer_state[0].bias = self.layer_state[0].bias - self.lr * self.layer_state[0].delta
        
        # update other weight and bias
        ## weight(i) = weight - learning * delta(i) * input(i - 1)
        ## bias(i) = bias - learning rate * delta(i) 
        i = 1    
        while i < np.size(self.layer_state):
            
            self.layer_state[i].weight = self.layer_state[i].weight - self.lr * self.layer_state[i].delta.dot( self.layer_state[i-1].output.T)
            self.layer_state[i].bias = self.layer_state[i].bias - self.lr * self.layer_state[i].delta
            i += 1
    
    def backpropagation(self, dataset):
        
        output = np.array(dataset[0,self.layer_state[0].inputs:])
        inputs = np.array(dataset[0,0:self.layer_state[0].inputs])

        self.calculate_delta(output.T)
        
        self.update_weights(inputs)
    
    def train_accuracy(self, output, ans):
        
        if np.array_equal(np.where(output == output.max()), np.where(ans == ans.max())) :
            
            return 1
        
        return 0
    
    def train(self, inputs, validation_rate, epoch):
        
        train_size = int(np.size(inputs, 0) * (1 - validation_rate))
        traininig_data =  np.array(inputs[ 0 : train_size ])
        validation_data = np.array(inputs[ train_size :   ])
        
        for i in range(epoch):
            
            train_accurate_counter = 0
            
            for j in range(train_size):
                
                output = self.feedforward(traininig_data[ j ])
                train_accurate_counter += self.train_accuracy(output, np.array(traininig_data[ j, self.layer_state[0].inputs : ]))
                self.backpropagation(np.array(traininig_data[ j ]))
                
            print("i :", i)
            print("aauracy : ", train_accurate_counter / train_size )
            

In [18]:
class Layer_state:
    
    def __init__(self, input_size, neuron_size, af):
        
        self.__neuron_size = neuron_size
        self.__input_size = input_size
        self.__weight = np.random.randn(neuron_size, input_size) 
        self.__bias = np.random.randn(neuron_size,1) 
        self.__af = af
        self.__output = np.array([])
        self.__delta = np.array([])
        
    @property
    def delta(self):
        
        return self.__delta
    
    @delta.setter
    def delta(self, delta):
        
        self.__delta = delta
    
    @property
    def inputs(self):
        
        return self.__input_size
    
    @property
    def neurons(self):
        
        return self.__neuron_size
    
    @property
    def weight(self):
        
        return self.__weight
    
    @weight.setter
    def weight(self, weight):
        
        self.__weight = weight
    
    @property
    def bias(self):
        
        return self.__bias
    
    @bias.setter
    def bias(self, bias):
        
        self.__bias = bias
    
    @property
    def output(self):
        
        return self.__output
    
    @output.setter
    def output(self, output):
    
        self.__output = output
    
    def feed_forward(self, inputs):
        
        n = self.weight.dot(inputs.T) + self.bias
        self.output = self.__af.func(n)        
        return self.__output.T

In [19]:
import pandas as pd
import numpy as np

train_data  = np.loadtxt('./train/train_img.txt', delimiter=',')
train_label = np.loadtxt('./train/train_label.txt', delimiter=',')
train_data  = np.array(train_data)
train_label = pd.get_dummies(train_label)

train = np.append(train_data, np.array(train_label), axis=1)
np.random.shuffle(train)
print(train)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [20]:
NN = nn(af='Sigmoid', ef='cross_entropy', lr=0.1)
NN.add_layer(np.size(train_data, 1), 32)
NN.add_layer(32, 3)

In [21]:
NN.train(train, 0, 3)

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   2.
 101. 228. 255. 151.  24.   4.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  69.
 253. 253. 253. 253. 253. 159.   2.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  32. 224.
 253. 253. 253. 253. 253. 253.  24.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  24. 202.
 253. 253. 194. 216. 253. 253.  24.   0.   0.   0.   0.   0.   0.   0.
   0. 

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed